# Twitter Data Search Application

In [5]:
# importing required libraries
import json
import psycopg2

# importing exceptions
from exceptions import *

In [ ]:
# connect to the databases for user and tweet information


### Information About the Databases
Fields in the user database:
* 'id'
* 'name'
* 'screen_name'
* 'location'
* 'created_at'
* 'followers_count'
* 'friends_count'      
* 'statuses_count'     (The total number of tweets that the account has posted.)
* 'favourites_count'   (The total number of tweets that the account has liked.)

### Functionalities of the Search App
The search app supports searches using usernames, screen names, words used in tweets and retweets (key-words) and geographical locations.
Users can query to get user information, like a user's screen name, their bio, follower count, their tweets, etc.
Users can also query on key-words to get tweets related to a certain topics. They can be ordered by date and popularity.

In [ ]:
def getUserInfo(userid):
    try:
        # search for the userid in the database
        # if found, pull up all the relevant info about the user
        pass
    except:
        raise UserNotFoundError

In [ ]:
def getTweetInfo(tweetid):
    try:
        # search for the tweetid
        # return the tweet, comments, retweet information, etc. if found in the database
        pass
    except:
        raise TweetNotFoundError

In [ ]:
def getKeywordTweets(keywords):
    keyword_list = keywords.split(",")
    for keyword in keywords:
        pass

In [ ]:
def search_twitter(userid = None, tweetid = None, keywords = None):
    """
        This function allows the end-user to query the database based on any of the specified input parameters.
        Multiple input parameters can also be specified for a more comprehensive search, if so desired.
    """
    if userid is not None:
        getUserInfo(userid)

    if tweetid is not None:
        getTweetInfo(tweetid)
    
    if keywords is not None:
        getKeywordTweets(keywords)